In [1]:
import numpy as np     
from scipy import signal
from tools import quick_scan, slow_scan
from multiprocessing import Pool

In [2]:
def generate_data(n_samps=20, n_chans=1):
    uV_scale = 25
    b, a = signal.butter(3, 0.5)
    data = uV_scale * np.random.randn(n_samps*n_chans).round(2) 
    data = signal.filtfilt(b, a, data)
    data = data.reshape(n_samps, n_chans)
    return data

# 1. Preparation

### Generate data

In [3]:
small_data = generate_data(n_samps=int(1e3), n_chans=1)
big_data = generate_data(n_samps=int(1e5), n_chans=32)

win_len = 30
max_excursion = 70

In [4]:
big_data_channels = [big_data[:, i] for i in range(big_data.shape[-1])]

# 2. Small data

### slow_scan

In [5]:
%%timeit
slow_scan(small_data, win_len, max_excursion)

12.3 ms ± 848 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### quick_scan

In [6]:
%%timeit
quick_scan(small_data, win_len, max_excursion)

518 µs ± 45.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Correctness check

In [7]:
slow_result = slow_scan(small_data, win_len, max_excursion)
quick_result = quick_scan(small_data, win_len, max_excursion)

(slow_result == quick_result).all()

True

# 3. Big data

### slow_scan

We only time one channel using slow_scan, since it's, well, slow.

In [8]:
channel_1 = big_data_channels[0]

In [9]:
%%timeit
slow_scan(channel_1, win_len, max_excursion)

1.34 s ± 50.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### quick_scan

Here's a run with channel_1:

In [10]:
%%timeit
quick_scan(channel_1, win_len, max_excursion)

77 ms ± 4.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Here's a run with all 32 channels:

In [11]:
%%timeit
for channel in big_data_channels:
    quick_scan(channel, win_len, max_excursion)

2.23 s ± 84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Correctness check

In [12]:
slow_result = slow_scan(channel_1, win_len, max_excursion)
quick_result = quick_scan(channel_1, win_len, max_excursion)

(slow_result == quick_result).all()

True

## Additional speedups can be achieved using multiprocessing:

In [13]:
arguments = [(channel, win_len, max_excursion) for channel in big_data_channels]

In [14]:
%%timeit
pool = Pool()
pool.starmap(quick_scan, arguments)
pool.close()
pool.join()

1.18 s ± 36.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The speedup is proportional to the number of cores.

### Correctness check

In [15]:
pool = Pool()
results = pool.starmap(quick_scan, arguments)
pool.close()
pool.join()

In [16]:
(results[0] == slow_result).all()

True